<a href="https://colab.research.google.com/github/Nish171/CS6910-Assignment-2/blob/main/Assignment_2B_Wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install wandb

     |████████████████████████████████| 2.1MB 20.9MB/s 
     |████████████████████████████████| 163kB 53.6MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 133kB 57.8MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=19ba5367d4fe6e9c2a6592deb1148678eb8fe0fb682e0dcac283a1355ccfffa4
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=0155d3f0d6f45ba6e244999b4075e43fba357ad232e9979c88f9aa037ea0cd0c
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [36]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
import wandb
from wandb.keras import WandbCallback

In [28]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception

In [2]:
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [30]:
sweep_config = {
  'name': 'sweep1b',  
  'method': 'grid',
  'parameters': {
        
        
        'fc_size':{
            'values':[64,128]
        },
        'batchnorm':{
            'values':['yes']
        },
        'augmentation':{
            'values':['yes']   
        },
        'droprate':{
            'values':[0.3,0.4]
        },
        'pre_train':{
            'values':['inceptionv3']
        }

           
    }

}

config_defaults={
    'fc_size':64,
    'batchnorm':'yes',
    'augmentation':'yes',
    'droprate':0.3,
    'pre_train':'inceptionv3'
}

In [41]:
def PreTrain():

  wandb.init(config=config_defaults)
  configs=wandb.config

  batchnorm=configs['batchnorm']
  droprate=configs['droprate']
  augmentation=configs['augmentation']
  fc_size=configs['fc_size']
  pre_train = configs['pre_train']

  wandb.run.name='model_'+pre_train+'_fc_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')


  if pre_train == 'inceptionv3':
    img_height = 299
    img_width = 299
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'inceptionresnetv2':
    img_height = 299
    img_width = 299
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'Xception':
    img_height = 299
    img_width = 299
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  


  for layers in base_model.layers:
    layers.trainable = False

  model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])
  if batchnorm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))
  


  augment= ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  train_set = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123,
  )

  aug_set = augment.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size=32,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
  )

  val_set = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='validation',
      seed=123,
  )

  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )

  if augmentation =='no' :
    hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])
  
  else:
    hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])
  val_acc=max(hist.history['val_accuracy'])
  params={'batch_norm':batchnorm,'augmentation':augmentation,'dropout':droprate,'pre_trained_model':pre_train,'val_acc':val_acc}
  wandb.log(params)

In [ ]:
sweep_id=wandb.sweep(sweep_config,entity='sanjeev-nimeesh',project='assignment_2')
wandb.agent(sweep_id, PreTrain)   

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: i39rtddw
Sweep URL: https://wandb.ai/sanjeev-nimeesh/assignment_2/sweeps/i39rtddw


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: fuwdkewu with config:
wandb: 	augmentation: yes
wandb: 	batchnorm: yes
wandb: 	droprate: 0.3
wandb: 	fc_size: 64
wandb: 	pre_train: inceptionv3


Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
161/282 [================>.............] - ETA: 1:57 - loss: 1.7115 - accuracy: 0.4731